In [ ]:
from sshtunnel import SSHTunnelForwarder

import psycopg2
import pandas as pd
import time


PORT=5432
REMOTE_SSH_PORT = 22

REMOTE_USERNAME = 'dvateam'
REMOTE_PASSWORD = 'DVA!Fall2021'
REMOTE_HOST = 'dvavm.eastus.cloudapp.azure.com'

DB_USERNAME = 'dva_readonly'
DB_PASSWORD = 'DVAFall2021'
DB_NAME = 'nyc-taxi-data'

server = SSHTunnelForwarder((REMOTE_HOST, REMOTE_SSH_PORT),
         ssh_username=REMOTE_USERNAME,
         ssh_password=REMOTE_PASSWORD,
         remote_bind_address=('localhost', PORT),
         local_bind_address=('localhost', 8080)
         
         )
server.start()
conn = psycopg2.connect(
user = DB_USERNAME,  host=server.local_bind_host,
   port=server.local_bind_port, dbname=DB_NAME, password=DB_PASSWORD)

cur = conn.cursor()


def create_pandas_table(sql_query, database = conn):
    table = pd.read_sql_query(sql_query, database)
    return table

#vendor_info = create_pandas_table("SELECT * FROM cab_types")
#print (vendor_info)

#vendor_info = create_pandas_table("SELECT cab_type_id,vendor_id,pickup_datetime, dropoff_datetime,passenger_count, trip_distance , fare_amount, payment_type, trip_type  from trips where pickup_datetime between '2020-01-01' and '2020-01-03' LIMIT 10000 ;")
#print (vendor_info)

#vendor_info = create_pandas_table("SELECT *  from fhv_trips where pickup_datetime between '2020-01-01' and '2020-01-02' LIMIT 10000 ;")
#print (vendor_info)

#vendor_info = create_pandas_table("SELECT count(1)  from trips where pickup_datetime between '2019-10-01' and '2019-12-31';")
#vendor_info = create_pandas_table("SELECT min(pickup_datetime) from trips")
#vendor_info = create_pandas_table("SELECT count(1)  from trips where date_part('Year',pickup_datetime) = 2019;")

#01-01 - 03-31 : 24,087,560
#04-01 - 06-30 : 23,249,433
#07-01 - 09-30 : 20,126,438
#10-01 - 12-31 : 22,181,065

query = '''
SELECT pickup_zone, dropoff_zone, month, cab_type_id, day_night, day_of_week,
       AVG(total_amount) as avg_total_amount, AVG(trip_time) as avg_trip_time, 
       COUNT(*) as num_trips
FROM         
    (SELECT pickup_location_id AS pickup_zone, 
           dropoff_location_id AS dropoff_zone,
           date_part('month', pickup_datetime) AS month,
           cab_type_id, 
           (fare_amount + tip_amount) AS total_amount, 
           ((date_part('epoch', dropoff_datetime) - 
            date_part('epoch', pickup_datetime))/3600)
               AS trip_time,
           CASE WHEN date_part('hour', pickup_datetime) >=6 AND 
                     date_part('hour', pickup_datetime) <18
                THEN 'day'
                ELSE 'night'
           END   
               AS day_night,
           CASE WHEN date_part('isodow', pickup_datetime) < 6
                THEN 'weekday'
                ELSE 'weekend'
           END
               AS day_of_week
       FROM trips
       WHERE pickup_location_id IS NOT NULL 
           AND dropoff_location_id IS NOT NULL
           AND pickup_location_id < 264
           AND dropoff_location_id < 264
           AND (fare_amount + tip_amount) > 2
           AND date_part('year', pickup_datetime)=2016 
           ) ta      
    WHERE trip_time > 0.08
        AND trip_time < 3
GROUP BY pickup_zone, dropoff_zone, month, cab_type_id, day_night, day_of_week
'''
start = time.time()

df = create_pandas_table(query)
print (df.head())
print(time.time() - start)
df.to_csv('TAXI_trips_2016.csv', index = False)
cur.close()
conn.close()


cur.close()
conn.close()